## 1. 실시간

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *
import datetime

class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Real")
        self.setGeometry(300, 300, 300, 400)

        # Button
        btn = QPushButton("구독", self)
        btn.move(20, 20)
        btn.clicked.connect(self.btn_clicked)

        btn2 = QPushButton("해지", self)
        btn2.move(180, 20)
        btn2.clicked.connect(self.btn2_clicked)
        
        # ocx 이벤트
        self.ocx = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.ocx.OnEventConnect.connect(self._handler_login)
        self.ocx.OnReceiveRealData.connect(self._handler_real_data)
        
        QTimer.singleShot(2000, self.CommmConnect)

    def btn_clicked(self):
        # self.SetRealReg("1000", "005930", "20;10", 0) # 삼성전자,  체결시간, 현재가 구독 신청
        self.SetRealReg("1000", "005930", "27;28", 0)
        # self.SetRealReg("2000", "", "215;20;214", 0)
        print("called\n")

    def btn2_clicked(self):
        self.DisConnectRealData("1000")

    def CommmConnect(self):
        self.ocx.dynamicCall("CommConnect()")
        self.statusBar().showMessage("login 중 ...")

    def _handler_login(self, err_code):
        if err_code == 0:
            self.statusBar().showMessage("login 완료")

    def _handler_real_data(self, code, real_type, data):
        print(code, real_type, data)
        if real_type == "장시작시간":
            gubun =  self.GetCommRealData(code, 215)
            remained_time =  self.GetCommRealData(code, 214)
            print(gubun, remained_time)
            
        elif real_type == "주식체결":
            # 체결 시간
            time = self.GetCommRealData(code, 20)
            date = datetime.datetime.now().strftime("%Y-%m-%d ")
            time = datetime.datetime.strptime(date + time, "%Y-%m-%d %H%M%S")
            print(time, end=" ")
            
            # 현재가
            price = self.GetCommRealData(code, 10)
            print(int(price))
            
        elif real_type == "주식우선호가":
            now = datetime.datetime.now()
            ask01 = self.GetCommRealData(code, 27)
            bid01 = self.GetCommRealData(code, 28)
            
            print(f"현재시간 {now} | 최우선매도호가: {ask01} 최우선매수호가: {bid01}")

    # 실시간 API 등록
    def SetRealReg(self, screen_no, code_list, fid_list, real_type):
        self.ocx.dynamicCall("SetRealReg(QString, QString, QString, QString)", screen_no, code_list, fid_list, real_type)
        self.statusBar().showMessage("구독 신청 완료")
        
    # 실시간 API 해제
    def DisConnectRealData(self, screen_no):
        self.ocx.dynamicCall("DisConnectRealData(QString)", screen_no)
        self.statusBar().showMessage("구독 해지 완료")

    # 이벤트 발생 시 데이터 수신
    def GetCommRealData(self, code, fid):
        data = self.ocx.dynamicCall("GetCommRealData(QString, int)", code, fid) 
        return data

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MyWindow()
    window.show()
    app.exec_()
    

called

